# Pokémon competition

In this notebook you have to provide the best pipeline that you have found to predict Pokémon battles.

At the end you will have to generate a set of predictions over the unlabeled data `data.hidden` and `data_inverse.hidden`. In these unlabeled dataset you will find all the Pokémon battles that we will be performing in some *fictional* Pokémon competition, so we do not know the outcome of these battles right now!

Remember to use all the tools that we have seen in class to evaluate and fine-tune your pipeline.

*Gotta Predict 'Em All!*

Paste here your pipeline:

In [1]:
# Data: Replace it with the data from the provided datasets
from pathlib import Path
import pandas

__wd__ = Path("__file__").resolve().parent
datasets_path = __wd__ / "datasets"

data = pandas.read_csv(datasets_path / "data.train", index_col=0)
inverse_data = pandas.read_csv(datasets_path / "data_inverse.train", index_col=0)

def get_Xy(dataset):
    return dataset.drop("Wins", axis=1), dataset["Wins"]

X, y = get_Xy(data)


# Pipeline: Replace it with the pipeline created in the other notebook.
# You can remove the import after replacing the pipeline.
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
pipeline = DummyClassifier(strategy="uniform")

num = ["HP","Attack", "Defense", "Sp. Atk","Sp. Def", "Speed","HP__other", "Attack__other","Defense__other", "Sp. Atk__other", "Sp. Def__other", "Speed__other" ]
cat = ['Type 1', 'Type 2', 'Type 1__other', 'Type 2__other']

pipeline = Pipeline(
    [
        (
            "preprocessing", ColumnTransformer(
                [
                    (
                        "cat",
                        make_pipeline(
                            SimpleImputer(strategy="most_frequent"),
                            OneHotEncoder(sparse=False, drop="first"),
                        ),
                        cat
                    ),
                    ("num", SimpleImputer(strategy="mean"), num),
                ],
                remainder="drop",
            )
        ),
        ("predictor", RandomForestClassifier(bootstrap=False, max_features=0.3, min_samples_split=3, n_estimators=300)),
    ]
)

# Train the pipeline
pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['Type 1', 'Type 2',
                                                   'Type 1__other',
                                                   'Type 2__other']),
                                                 ('num', SimpleImputer(),
                                                  ['HP', 'Attack', 'Defense',
                                                   'Sp. Atk', 'Sp. Def',
    

In [32]:
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!
import pandas
from pathlib import Path

__wd__ = Path("__file__").resolve().parent
datasets_path = __wd__ / "datasets"

tournament = pandas.read_csv(datasets_path / "data.hidden", index_col=0)
tournament_inverse = pandas.read_csv(datasets_path / "data_inverse.hidden", index_col=0)

y_predicted = pipeline.predict(tournament)
y_inverse_predicted = pipeline.predict(tournament_inverse)

y_predicted.tofile("predicted.csv", sep=",")
y_inverse_predicted.tofile("predicted_inverse.csv", sep=",")
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!